In [39]:
import pandas as pd
import numpy as np
import re
import sys  
sys.path.insert(0, '/home/jovyan/work/chatbot-trainer/modules/')
import bfs

In [41]:
visited = [] # List to keep track of visited nodes.
queue = []     #Initialize a queue

def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0) 
    print (s, end = " ") 

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)


In [2]:
df = pd.read_csv("../data/twcs/twcs.csv")

In [3]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
len(df[df.author_id == '115712']['text'].tolist())

11

In [5]:
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound][df.author_id == 'AppleSupport']

# Get replies for first inbound tweets
inbound_and_replies = pd.merge(first_inbound, df, left_on='tweet_id', right_on='in_response_to_tweet_id')

# Remove replies that aren't from companies
inbound_and_replies = inbound_and_replies[inbound_and_replies.inbound_y ^ True]

# Replace carriage returns
inbound_and_replies = inbound_and_replies.replace('\r', '', regex=True)

<ipython-input-5-de960f4c1846>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound][df.author_id == 'AppleSupport']


In [6]:
tweets = inbound_and_replies[['text_x', 'text_y']]
tweets.columns = ['tweet', 'response']
tweets.head(50)

,tweet,response


In [7]:
len(pd.unique(df.author_id))

702777

In [8]:
pd.unique(df.author_id)

array(['sprintcare', '115712', '115713', ..., '823868', '823869',
       '823870'], dtype=object)

In [9]:
pd.unique(df[df.inbound == False].author_id)
# if company's 

array(['sprintcare', 'Ask_Spectrum', 'VerizonSupport', 'ChipotleTweets',
       'AskPlayStation', 'marksandspencer', 'MicrosoftHelps',
       'ATVIAssist', 'AdobeCare', 'AmazonHelp', 'XboxSupport',
       'AirbnbHelp', 'nationalrailenq', 'AirAsiaSupport', 'Morrisons',
       'NikeSupport', 'AskAmex', 'McDonalds', 'YahooCare', 'AskLyft',
       'UPSHelp', 'Delta', 'AppleSupport', 'Uber_Support', 'Tesco',
       'SpotifyCares', 'British_Airways', 'comcastcares', 'AmericanAir',
       'TMobileHelp', 'VirginTrains', 'SouthwestAir', 'AskeBay',
       'hulu_support', 'GWRHelp', 'sainsburys', 'AskPayPal', 'HPSupport',
       'ChaseSupport', 'CoxHelp', 'DropboxSupport', 'VirginAtlantic',
       'BofA_Help', 'AzureSupport', 'AlaskaAir', 'ArgosHelpers',
       'Postmates_Help', 'AskTarget', 'GoDaddyHelp', 'CenturyLinkHelp',
       'AskPapaJohns', 'SW_Help', 'askpanera', 'Walmart',
       'USCellularCares', 'AsurionCares', 'GloCare', 'idea_cares',
       'DoorDash_Help', 'NeweggService', 'VirginA

In [10]:
df.head(20)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


In [18]:
# To Do
# 1. Sanitize data by removing user names from text
# df['text_nouser'] = df['text'].apply(lambda x: re.sub('@[A-Za-z0-9]+' , '', str(x.lstrip())))
df['text_nouser'] = df['text'].apply(lambda x: re.sub('@[^\s]+' , '', str(x.lstrip())))
df['text_nouser']

0           I understand. I would like to assist you. We ...
1                          and how do you propose we do that
2           I have sent several private messages and no o...
3           Please send us a Private Message so that we c...
4                                                     I did.
                                 ...                        
2811769     Hey, we'd be happy to look into this for you....
2811770     wtf!? I’ve been having really shitty service ...
2811771      You have to go to https://t.co/v2tMHeTL7Q an...
2811772     Sounds delicious, Sarah! 😋 https://t.co/7uQPw...
2811773      warm sloe gin mince pies with ice cream - th...
Name: text_nouser, Length: 2811774, dtype: object

In [19]:
# 2. Find similarities between conversations of different companies (word2vec)


In [20]:
# 3. Find distribution between corp and non corp.


In [21]:
# 4. Find length of the conversation
df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))
df[['text','word_count']].head(5)

,text,word_count
0,@115712 I understand. I would like to assist y...,23
1,@sprintcare and how do you propose we do that,9
2,@sprintcare I have sent several private messag...,14
3,@115712 Please send us a Private Message so th...,23
4,@sprintcare I did.,3


In [22]:
# 5. Most common words in question, answers, back and forth conversation


In [23]:
# 6. Size of the entire and unique words


In [45]:
# 7. Unique conversation beginning
conversation_starter = df[df.in_response_to_tweet_id.isna() & df.inbound]
conversation_thread  = pd.merge(conversation_starter, df, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')
conversation_thread.groupby(["author_id_x"])

In [26]:
bfs.bfs()

AttributeError: module 'bfs' has no attribute 'bfs'

In [15]:
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]

In [160]:
df.groupby(df.author_id).size().reset_index()

,author_id,0
0,10026,3
1,100363,1
2,10103,2
3,10221,2
4,10286,1
...,...,...
702772,mediatemplehelp,305
702773,nationalrailenq,4444
702774,sainsburys,19466
702775,sizehelpteam,1482


In [161]:
df.groupby(df.inbound).size().reset_index()

,inbound,0
0,False,1273931
1,True,1537843
